In [9]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

import matplotlib.pyplot as plt
import random
import os

In [10]:
def plt_display(image, title):
    fig = plt.figure()
    a = fig.add_subplot(1, 1, 1)
    imgplot = plt.imshow(image)
    a.set_title(title)

## Build Source Domain Network

In [11]:
from models import encoder, unet

s_encoder = encoder.get_model()
u_net = unet.get_model()
s_model = keras.models.Model(
    inputs=s_encoder.inputs,
    outputs=u_net(s_encoder(s_encoder.inputs))
)

2022-07-02 21:54:34.956823: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-02 21:54:34.959952: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-07-02 21:54:34.991653: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-02 21:54:34.991791: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: Quadro T1000 computeCapability: 7.5
coreClock: 1.455GHz coreCount: 14 deviceMemorySize: 3.82GiB deviceMemoryBandwidth: 104.34GiB/s
2022-07-02 21:54:34.991809: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-07-02 21:54:34.998771: I tensorflow/stream_executor/platform/default/dso_l

## Load Zurich dataset as source dataset
- https://docs.google.com/forms/d/e/1FAIpQLSdH6Pqdlu0pk2vGZlazqoRYwWsxN3nsLFwYY6Zc5-RUjw3SdQ/viewform

In [15]:
#base_dir = "/home/gosha20777/Documents/Datasets/Zurich-RAW-to-DSLR-Dataset"
base_dir = "/home/gosha20777/Zurich-RAW-to-DSLR-Dataset"
train_input_dir = os.path.join(base_dir, "train/huawei_raw")
train_target_dir = os.path.join(base_dir, "train/canon")
test_input_dir = os.path.join(base_dir, "test/huawei_raw")
test_target_dir = os.path.join(base_dir, "test/canon")
img_size = (224, 224)
dlsr_scale = 2
input_channels = 4
output_channels = 3
batch_size = 8

input_img_paths = sorted(
    [
        os.path.join(train_input_dir, fname)
        for fname in os.listdir(train_input_dir)
        if fname.endswith(".png")
    ]
)
target_img_paths = sorted(
    [
        os.path.join(train_target_dir, fname)
        for fname in os.listdir(train_target_dir)
        if fname.endswith(".jpg")
    ]
)

test_input_img_paths = sorted(
    [
        os.path.join(test_input_dir, fname)
        for fname in os.listdir(test_input_dir)
        if fname.endswith(".png")
    ]
)
test_target_img_paths = sorted(
    [
        os.path.join(test_target_dir, fname)
        for fname in os.listdir(test_target_dir)
        if fname.endswith(".jpg")
    ]
)


print("Number of samples:", len(input_img_paths))
print("Number test of samples:", len(test_input_img_paths))

for input_path, target_path in zip(input_img_paths[:10], target_img_paths[:10]):
    print(os.path.basename(input_path), "|", os.path.basename(target_path))

Number of samples: 46839
Number test of samples: 1204
0.png | 0.jpg
1.png | 1.jpg
10.png | 10.jpg
100.png | 100.jpg
1000.png | 1000.jpg
10000.png | 10000.jpg
10001.png | 10001.jpg
10002.png | 10002.jpg
10003.png | 10003.jpg
10004.png | 10004.jpg


In [25]:
from datasets.huavei import HuaveiLoader

val_samples = 4000
random.Random(42).shuffle(input_img_paths)
random.Random(42).shuffle(target_img_paths)
train_input_img_paths = input_img_paths[:-val_samples]
train_target_img_paths = target_img_paths[:-val_samples]
val_input_img_paths = input_img_paths[-val_samples:]
val_target_img_paths = target_img_paths[-val_samples:]

# Instantiate data Sequences for each split
train_gen = HuaveiLoader(
    batch_size, img_size, dlsr_scale, train_input_img_paths, train_target_img_paths
)
val_gen = HuaveiLoader(
    batch_size, img_size, dlsr_scale, val_input_img_paths, val_target_img_paths
)
test_gen = HuaveiLoader(
    batch_size, img_size, dlsr_scale, test_input_img_paths, test_target_img_paths
)

In [18]:
x, y = train_gen.__getitem__(0)
plt_display(x[3][:,:,0:3], 'input')
plt_display(y[3], 'target')

Process ForkPoolWorker-41:
Traceback (most recent call last):
  File "/home/gosha20777/anaconda3/envs/nir-3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/gosha20777/anaconda3/envs/nir-3/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/gosha20777/anaconda3/envs/nir-3/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/home/gosha20777/anaconda3/envs/nir-3/lib/python3.9/multiprocessing/queues.py", line 368, in get
    return _ForkingPickler.loads(res)
AttributeError: 'HuaveiLoader' object has no attribute '__read_element_multi_run_wrapper'
Process ForkPoolWorker-42:
Traceback (most recent call last):
  File "/home/gosha20777/anaconda3/envs/nir-3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/gosha20777/anaconda3/envs/nir-3/lib/python3.9/multiprocessing/process.py", line 108, in run
    se

KeyboardInterrupt: 

## Train Source Model

In [9]:
from losses.losses import mssim, vgg_loss

def main_loss(y_true, y_pred):
    return tf.keras.losses.mean_absolute_error(y_true, y_pred) + \
        mssim(y_true, y_pred) + \
        vgg_loss(y_true, y_pred)

s_model.compile(optimizer="adam", loss=main_loss)

In [11]:
epochs = 4

callbacks = [
    keras.callbacks.ModelCheckpoint("s_model.h5", save_best_only=True)
]


s_model.fit(train_gen, epochs=epochs, validation_data=val_gen, callbacks=callbacks, workers=8)

Epoch 1/4
 310/5354 [>.............................] - ETA: 31:24 - loss: 0.8947

KeyboardInterrupt: 

In [16]:
import multiprocessing

In [45]:
def multi_run_wrapper(args):
   return add(*args)

def add(x, y):
    return x, y

In [48]:
a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
b = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

with multiprocessing.Pool(2) as pool:
    results = pool.map(multi_run_wrapper, zip(a, b))

results
for j, (x, y) in enumerate(results):
    print(x)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [37]:
for i in zip(a, b):
    print(i)

(1, 1)
(2, 2)
(3, 3)
(4, 4)
(5, 5)
(6, 6)
(7, 7)
(8, 8)
(9, 9)
(10, 10)
(11, 11)
(12, 12)
(13, 13)
(14, 14)
(15, 15)
(16, 16)
(17, 17)
(18, 18)
(19, 19)
(20, 20)
